In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt
from lmfit import models
from bokeh.plotting import figure, output_notebook, show

/home/frederik/anaconda3/lib/python3.4/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


In [2]:
pc_henrik = '/home/arrow13/Git/FP/Roentgen/'
pc_frederik = '/home/frederik/Dokumente/FP/Roentgen'

In [3]:
var_pc = pc_frederik

In [4]:
cd $var_pc/Messdaten

/home/frederik/Dokumente/FP/Roentgen/Messdaten


In [5]:
data = np.loadtxt("Si111_40_30.xy").T

In [6]:
plt.plot(data[0],data[1])
plt.show()

# Einlesen, vorbereiten und auswerten der Daten (mit Versuchsplan)

## Messen des Röntgenspektrums der Kupferanode
Es werden drei verschiedene Spannungen und Ströme verwendet
+ U$_1$ = 30keV, A$_1$ = 10mA
+ U$_2$ = 40keV, A$_2$ = 10mA
+ U$_3$ = 40keV, A$_3$ = 30mA

Die letzte Einstellung wird noch mal mit einem Ni-Filter wiederholt

+ U$_{Ni}$ = 30keV, A$_{Ni}$ = 30mA

Auswertung:
+ Zählrate als Funktion des Winkels auftragen
+ Die lage der K$_{\alpha_{1,2}}$- und der K$_\beta$-Linien bestimmen, sowie die Verhältnisse (unter einander und unter den Ordnungen)
+ Mit Ni-Filter die Abschwächung der K$_\beta$-Linie, das "Signal-zu-Rausch" Verhältnis für die K$_{\alpha_{1,2}}$-Linie, die Energie und die Energiebreite bestimmen
+ Weitere Details des Spektrums diskutieren

## Netzebenabstand Si(331)
+ Aus dem Diffraktogramm soll der Netzebenabstand von Si(331) bestimmt werden
+ Der bestimmte Netzebenabstand soll mit Literaturwerten abgeglichen werden

## Netzebenabstand Ge(111)
+ Aus dem Diffraktogramm soll der Netzebenabstand von Si(331) bestimmt werden
+ Der bestimmte Netzebenabstand soll mit Literaturwerten abgeglichen werden

## Pulverdiraktometrie
+ Aus dem Diffraktogramm soll mittels einer Datenbank qualitativ die Zusammensetzung bestimmt werden
+ Unabhängig von der Datenbank sollen die Netzebenabstände des Pulvers bestimmt werden
+ Graphisch zeigen, dass die gefundene Kirstallstrucktur mit dem Diffraktogramm verträglich ist
+ Die mittlere Kristallgröße ermitteln

### Voigt für den einzel Peak 

In [7]:
intervall_voigt_1 = np.arange(np.where(data[0] > 83.43)[0][0], np.where(data[0] > 83.5862)[0][0])
err_1 = np.sqrt(data[1][intervall_voigt_1])
err_1 = np.where(err_1 == 0, 1, err_1)

In [8]:
voigt_1 = models.VoigtModel(prefix='voigt_1')
pars_1 = voigt_1.guess(data[1][intervall_voigt_1],x=data[0][intervall_voigt_1])
pars_1['voigt_1gamma'].set(0.001,vary=True,expr='')

In [9]:
out_1 = voigt_1.fit(data[1][intervall_voigt_1],pars_1,x=data[0][intervall_voigt_1], weights=1/(err_1))

In [10]:
out_1.plot()
plt.errorbar(data[0][intervall_voigt_1],data[1][intervall_voigt_1],err_1,fmt='. ')
plt.show()

In [11]:
print(out_1.fit_report())

[[Model]]
    Model(voigt, prefix='voigt_1')
[[Fit Statistics]]
    # function evals   = 31
    # data points      = 37
    # variables        = 4
    chi-square         = 68.009
    reduced chi-square = 2.061
[[Variables]]
    voigt_1amplitude:   184.071531 +/- 3.630029 (1.97%) (init= 112.6005)
    voigt_1sigma:       0.01169932 +/- 0.003771 (32.24%) (init= 0.02172465)
    voigt_1center:      83.5109857 +/- 0.000432 (0.00%) (init= 83.51311)
    voigt_1gamma:       0.03792108 +/- 0.002355 (6.21%) (init= 0.001)
    voigt_1fwhm:        0.04213289 +/- 0.013582 (32.24%)  == '3.6013100*voigt_1sigma'
[[Correlations]] (unreported correlations are <  0.100)
    C(voigt_1sigma, voigt_1gamma)  = -0.956 
    C(voigt_1amplitude, voigt_1gamma)  =  0.908 
    C(voigt_1amplitude, voigt_1sigma)  = -0.848 


### Mulit-Voigt

In [12]:
intervall_voigt_2 = np.arange(np.where(data[0] > 94.875)[0][0], np.where(data[0] > 95.3)[0][0])
guess_intervall_voigt_2 = np.arange(np.where( 94.875 < data[0])[0][0],np.where( data[0] > 95.1)[0][0])
guess_intervall_voigt_3 = np.arange(np.where( 95.1 < data[0])[0][0],np.where( data[0] > 95.3)[0][0])
err_2 = np.sqrt(data[1][intervall_voigt_2])
err_2 = np.where(err_2 == 0, 1, err_2)

In [13]:
voigt_2 = models.VoigtModel(prefix='voigt_2')
voigt_3 = models.VoigtModel(prefix='voigt_3')
dual_voigt = voigt_2 + voigt_3
pars_2 = voigt_2.guess(data[1][guess_intervall_voigt_2],x=data[0][guess_intervall_voigt_2])
pars_2.update(voigt_3.guess(data[1][guess_intervall_voigt_3],x=data[0][guess_intervall_voigt_3]))
pars_2['voigt_2gamma'].set(0.0001,vary=True,expr='')
pars_2['voigt_3gamma'].set(0.0001,vary=True,expr='')
pars_2['voigt_2amplitude'].set(1000)
pars_2['voigt_3amplitude'].set(1000)

In [14]:
out_2 = dual_voigt.fit(data[1][intervall_voigt_2],pars_2,x=data[0][intervall_voigt_2],weights=1/(err_2))

In [15]:
out_2.plot_fit()
plt.errorbar(data[0][intervall_voigt_2],data[1][intervall_voigt_2],err_2,fmt='. ')
plt.show()

In [16]:
print(out_2.fit_report())

[[Model]]
    (Model(voigt, prefix='voigt_2') + Model(voigt, prefix='voigt_3'))
[[Fit Statistics]]
    # function evals   = 55
    # data points      = 102
    # variables        = 8
    chi-square         = 399.960
    reduced chi-square = 4.255
[[Variables]]
    voigt_2sigma:       0.01150577 +/- 0.000814 (7.08%) (init= 0.01629349)
    voigt_2amplitude:   710.601029 +/- 5.023346 (0.71%) (init= 1000)
    voigt_2center:      94.9398390 +/- 0.000207 (0.00%) (init= 94.93944)
    voigt_2gamma:       0.02417442 +/- 0.000550 (2.28%) (init= 0.0001)
    voigt_2fwhm:        0.04143587 +/- 0.002932 (7.08%)  == '3.6013100*voigt_2sigma'
    voigt_3sigma:       0.01825475 +/- 0.001054 (5.77%) (init= 0.01900907)
    voigt_3center:      95.2518722 +/- 0.000345 (0.00%) (init= 95.25278)
    voigt_3amplitude:   362.023717 +/- 4.443356 (1.23%) (init= 1000)
    voigt_3gamma:       0.02132134 +/- 0.001044 (4.90%) (init= 0.0001)
    voigt_3fwhm:        0.06574102 +/- 0.003796 (5.77%)  == '3.6013100*voigt_3

In [17]:
plt.subplot(121)
plt.errorbar(data[0][intervall_voigt_1],data[1][intervall_voigt_1],err_1,fmt='. ',label="Messdaten")
plt.plot(data[0][intervall_voigt_1],out_1.best_fit,label="Voigt-Fit")
plt.ticklabel_format(useOffset=False)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best',fontsize=42)
plt.xlabel('Winkel in 2$\Theta$ /$^\circ$',fontsize=42)
plt.ylabel('Counts',fontsize=42)
plt.title('Fit für den einzelnen Peak',fontsize=42)


plt.subplot(122)
plt.errorbar(data[0][intervall_voigt_2],data[1][intervall_voigt_2],err_2,fmt='. ',label="Messdaten")
plt.plot(data[0][intervall_voigt_2],out_2.best_fit,label="Multi-Voigt-Fit")
plt.ticklabel_format(useOffset=False)
plt.xlabel('Winkel in 2$\Theta$ /$^\circ$',fontsize=42)
plt.ylabel('Counts',fontsize=42)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.legend(loc='best',fontsize=42)
plt.title('Fit für die überlagerten Peaks',fontsize=42)

plt.show()

# Plots der Spektren für die ersten 4 Aufnahme

In [18]:
data_spektrum = np.loadtxt("Si111_40_30_Ni.xy").T

# Experimental interactiv Bokeh Plot

In [19]:
def err(a,d_a,b,d_b):
    print(a/b, np.sqrt((d_a/b)**2 + ((d_b * a)/(b**2))**2))

In [35]:
err(425,6,498,158)

0.8534136546184738 0.271029685939


In [36]:
err(1149,52,1046,873)

1.098470363288719 0.918139049885


In [37]:
err(654,75,1814,613)

0.360529217199559 0.128656943469


In [38]:
err(57,2,184,4)

0.30978260869565216 0.0127866982635


In [39]:
err(245,2,711,6)

0.34458509142053445 0.00404579525062


In [40]:
err(122,2,362,4)

0.3370165745856354 0.00666271928547
